# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

/home/shashank/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shashank/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/shashank/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shashank/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shashank/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size ch

In [6]:
hook = sy.TorchHook(th)

<module 'torch' from '/home/shashank/anaconda3/lib/python3.6/site-packages/torch/__init__.py'>

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{50844634909: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #tensors:1>

In [14]:
x.id_at_location

50844634909

In [15]:
x.id

85673375777

In [16]:
x.owner

<VirtualWorker id:me #tensors:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #tensors:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:85673375777 -> bob:50844634909]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [11]:
# try this project here!
import syft as sf
import torch as th

In [14]:
hook=sy.TorchHook(th)

In [15]:
a=th.tensor([1,2,3,4,5])

In [16]:
bob=sy.VirtualWorker(hook,id="bob")

In [17]:
alice=sy.VirtualWorker(hook,id="alice")

In [18]:
x=a.send(bob,alice)

In [19]:
x

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:85938923241 -> bob:63018984679]
	-> (Wrapper)>[PointerTensor | me:52642306976 -> alice:23464238629]

In [21]:
bob._objects

{63018984679: tensor([1, 2, 3, 4, 5])}

In [22]:
alice._objects

{23464238629: tensor([1, 2, 3, 4, 5])}

In [25]:
x=x.get()

In [26]:
x

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

# Lesson: Introducing Remote Arithmetic

In [27]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [28]:
x

(Wrapper)>[PointerTensor | me:7344279461 -> bob:74976771769]

In [29]:
y

(Wrapper)>[PointerTensor | me:68830170639 -> bob:70981483926]

In [30]:
z = x + y

In [31]:
z

(Wrapper)>[PointerTensor | me:79875157913 -> bob:79875157913]

In [32]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [33]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:28437210120 -> bob:28437210120]

In [34]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [35]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [36]:
z = (x + y).sum()

In [37]:
z.backward()

In [38]:
x = x.get()

In [39]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [40]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [115]:
# try this project here!
import torch as th
import syft as sf
import numpy as np

In [116]:
hook=sf.TorchHook(th)
bob=sf.VirtualWorker(hook,id="bob")

In [117]:
features=th.rand((5,2),requires_grad=True).send(bob)

In [118]:
weights=th.rand((2,1),requires_grad=True).send(bob)
bias=th.rand(1).send(bob)

In [119]:
target=th.rand((5,1),requires_grad=True).send(bob)


In [121]:
for i in range(10):
    pred=features.mm(weights)+bias
    loss=((target-pred)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad*0.1)
    weights.grad *=0
    print(loss.get())

tensor(0.3277, requires_grad=True)
tensor(0.3228, requires_grad=True)
tensor(0.3184, requires_grad=True)
tensor(0.3144, requires_grad=True)
tensor(0.3108, requires_grad=True)
tensor(0.3074, requires_grad=True)
tensor(0.3044, requires_grad=True)
tensor(0.3017, requires_grad=True)
tensor(0.2992, requires_grad=True)
tensor(0.2969, requires_grad=True)


In [101]:
loss

AttributeError: 'Tensor' object has no attribute 'child'

(Wrapper)>[PointerTensor | me:33566111392 -> bob:49204373497]

tensor(0.8650, requires_grad=True)


# Lesson: Garbage Collection and Common Errors


In [44]:
bob = bob.clear_objects()

In [45]:
bob._objects

{}

In [46]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [47]:
bob._objects

{23831414651: tensor([1, 2, 3, 4, 5])}

In [48]:
del x

In [49]:
bob._objects

{}

In [50]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
bob._objects

{92212512027: tensor([1, 2, 3, 4, 5])}

In [52]:
x = "asdf"

In [53]:
bob._objects

{}

In [54]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
x

(Wrapper)>[PointerTensor | me:19384969793 -> bob:49166137090]

In [56]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [57]:
x = "asdf"

In [58]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [59]:
del x

In [60]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [61]:
bob = bob.clear_objects()
bob._objects

{}

In [62]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [63]:
bob._objects

{17426510898: tensor([1, 2, 3, 4, 5])}

In [64]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [65]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

In [130]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [66]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [53]:
from torch import nn, optim

In [64]:
# A Toy Dataset
data = th.tensor([[0.8223,0.232],[0.233,0.43],[0.453,0.345],[0.23,0.123]], requires_grad=True)
target = th.tensor([[0.123],[0.12], [0.54], [0.677]], requires_grad=True)

In [84]:
data=th.rand([6,2],requires_grad=True)
target=th.rand([6,1],requires_grad=True)

In [85]:
# A Toy Model
model = nn.Linear(2,1)

In [86]:
target_bob.get()

tensor([[0.7193],
        [0.7540]], requires_grad=True)

In [87]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [88]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(5.0996)
tensor(2.9350)
tensor(1.7443)
tensor(1.0863)
tensor(0.7203)
tensor(0.5147)
tensor(0.3976)
tensor(0.3297)
tensor(0.2891)
tensor(0.2641)
tensor(0.2479)
tensor(0.2370)
tensor(0.2293)
tensor(0.2235)
tensor(0.2190)
tensor(0.2154)
tensor(0.2125)
tensor(0.2100)
tensor(0.2079)
tensor(0.2061)


In [89]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [90]:
data_alice = data[2:6].send(alice)
target_alice = target[2:6].send(alice)

In [91]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [100]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [101]:
train()

tensor(1.1985, requires_grad=True)
tensor(0.0466, requires_grad=True)
tensor(0.3105, requires_grad=True)
tensor(0.0481, requires_grad=True)
tensor(0.2917, requires_grad=True)
tensor(0.0443, requires_grad=True)
tensor(0.2807, requires_grad=True)
tensor(0.0435, requires_grad=True)
tensor(0.2744, requires_grad=True)
tensor(0.0440, requires_grad=True)
tensor(0.2708, requires_grad=True)
tensor(0.0450, requires_grad=True)
tensor(0.2687, requires_grad=True)
tensor(0.0460, requires_grad=True)
tensor(0.2675, requires_grad=True)
tensor(0.0469, requires_grad=True)
tensor(0.2668, requires_grad=True)
tensor(0.0477, requires_grad=True)
tensor(0.2664, requires_grad=True)
tensor(0.0482, requires_grad=True)
tensor(0.2662, requires_grad=True)
tensor(0.0486, requires_grad=True)
tensor(0.2661, requires_grad=True)
tensor(0.0489, requires_grad=True)
tensor(0.2660, requires_grad=True)
tensor(0.0490, requires_grad=True)
tensor(0.2660, requires_grad=True)
tensor(0.0491, requires_grad=True)
tensor(0.2660, requi

## My dataset

In [180]:
import torch
import syft as sf
from torch import optim,nn

In [181]:
data_1=torch.rand([1000,20],requires_grad=True)
labels_1=torch.rand([1000,3],requires_grad=True)


In [182]:
hook=sf.TorchHook(torch)
bob=sf.VirtualWorker(hook,id="bob")
alice=sf.VirtualWorker(hook,id="alice")

In [183]:
data_1[:500].shape

torch.Size([500, 20])

In [184]:
data_bob=data_1[:500].send(bob)
label_bob=labels_1[:500].send(bob)

In [185]:
data_alice=data_1[500:].send(alice)
label_alice=labels_1[500:].send(alice)

In [186]:
model=nn.Sequential(nn.Linear(10000,128),
                   )

In [187]:
criterion=nn.NLLLoss()

In [188]:
datasets=[(data_bob,label_bob),(data_alice,label_alice)]

In [189]:
def train(epochs,model,datasets):
    opt=optim.SGD(params=model.parameters(),lr=0.001)
    for e in range(epochs):
        running_loss=0
        for data,labels in datasets:
            data=data.view(data.shape[0],-1)
            model=model.send(data.location)
            opt.zero_grad()
            output=model(data)
            loss=criterion(output,labels)
            loss.backward()
            optim.step()
            model=model.get()
            print(loss.get())

In [190]:
train(10,model,datasets)

RuntimeError: size mismatch, m1: [500 x 20], m2: [10000 x 128] at /opt/conda/conda-bld/pytorch_1556653183467/work/aten/src/TH/generic/THTensorMath.cpp:961

In [195]:
data_bob.view(data_bob.shape[0], -1).get().shape


torch.Size([500, 20])

In [197]:
data_bob[1].shape

torch.Size([20])

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [163]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:bob #tensors:0>

<VirtualWorker id:alice #tensors:0>

In [167]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [170]:
x = x.send(alice)

In [171]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5])}

In [172]:
alice._objects

{55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [174]:
y = x + x

In [175]:
y

(Wrapper)>[PointerTensor | me:48924169518 -> alice:48924169518]

In [176]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5]),
 48924169518: tensor([ 2,  4,  6,  8, 10])}

In [177]:
alice._objects

{48924169518: (Wrapper)>[PointerTensor | alice:48924169518 -> bob:48924169518],
 55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [178]:
jon = sy.VirtualWorker(hook, id="jon")

In [186]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [187]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [188]:
alice._objects

{40541026396: (Wrapper)>[PointerTensor | alice:40541026396 -> bob:10539507281]}

In [189]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:40541026396 -> bob:10539507281]

In [190]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [191]:
alice._objects

{}

In [192]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [193]:
bob._objects

{}

In [194]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [195]:
bob._objects

{28843833290: tensor([1, 2, 3, 4, 5])}

In [196]:
alice._objects

{88814770977: (Wrapper)>[PointerTensor | alice:88814770977 -> bob:28843833290]}

In [197]:
del x

In [198]:
bob._objects

{}

In [199]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [212]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

In [213]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [214]:
bob._objects

{81966670653: tensor([1, 2, 3, 4, 5])}

In [215]:
alice._objects

{}

In [216]:
x.move(alice)

(Wrapper)>[PointerTensor | me:547575813 -> alice:547575813]

In [217]:
bob._objects

{}

In [218]:
alice._objects

{547575813: tensor([1, 2, 3, 4, 5])}

In [202]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [203]:
bob._objects

{75487377866: tensor([1, 2, 3, 4, 5])}

In [204]:
alice._objects

{94092707138: (Wrapper)>[PointerTensor | alice:94092707138 -> bob:75487377866]}

In [205]:
x.remote_get()

(Wrapper)>[PointerTensor | me:87205391815 -> alice:94092707138]

In [206]:
bob._objects

{}

In [207]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}

In [208]:
x.move(bob)

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [209]:
x

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [210]:
bob._objects

{87205391815: tensor([1, 2, 3, 4, 5])}

In [211]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}

### Project For Aggregated Federated Learning

In [1]:
import syft as sf
import torch as th
from torch import optim,nn

/home/shashank/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shashank/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/shashank/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shashank/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/shashank/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size ch

In [2]:
hook=sf.TorchHook(th)
bob=sf.VirtualWorker(hook,id="bob")
alice=sf.VirtualWorker(hook,id="alice")

In [3]:
features=th.rand([6,2],requires_grad=True)
target=th.rand([6,1],requires_grad=True)

In [4]:
data_bob=features[0:3].send(bob)
target_bob=target[0:3].send(bob)
data_alice=features[3:6].send(alice)
target_alice=target[3:6].send(alice)

In [5]:
datasets=[(data_bob,target_bob),(data_alice,target_alice)]

In [6]:
model = nn.Linear(2,1)
opt = optim.SGD(params=model.parameters(), lr=0.1)

for iter in range(20):

    for _data, _target in datasets:

        # send model to the data
        model = model.send(_data.location)

        # do normal training
        opt.zero_grad()
        pred = model(_data)
        loss = ((pred - _target)**2).sum()
        loss.backward()
        opt.step()

        # get smarter model back
        model = model.get()

        print(loss.get())

tensor(2.5114, requires_grad=True)
tensor(0.2105, requires_grad=True)
tensor(0.1091, requires_grad=True)
tensor(0.0351, requires_grad=True)
tensor(0.0932, requires_grad=True)
tensor(0.0335, requires_grad=True)
tensor(0.0848, requires_grad=True)
tensor(0.0323, requires_grad=True)
tensor(0.0780, requires_grad=True)
tensor(0.0315, requires_grad=True)
tensor(0.0725, requires_grad=True)
tensor(0.0310, requires_grad=True)
tensor(0.0679, requires_grad=True)
tensor(0.0307, requires_grad=True)
tensor(0.0641, requires_grad=True)
tensor(0.0306, requires_grad=True)
tensor(0.0610, requires_grad=True)
tensor(0.0306, requires_grad=True)
tensor(0.0585, requires_grad=True)
tensor(0.0306, requires_grad=True)
tensor(0.0563, requires_grad=True)
tensor(0.0308, requires_grad=True)
tensor(0.0546, requires_grad=True)
tensor(0.0310, requires_grad=True)
tensor(0.0531, requires_grad=True)
tensor(0.0312, requires_grad=True)
tensor(0.0518, requires_grad=True)
tensor(0.0314, requires_grad=True)
tensor(0.0508, requi

In [7]:
model=nn.Linear(2,1)
opt=optim.SGD(params=model.parameters(),lr=0.001)
    
for i in range(10):
    for _data,_labels in datasets:
        model=model.send(_data.location)
        opt.zero_grad()
        pred=model(_data)
        loss=((pred-_labels)**2).sum()
        loss.backward()
        opt.step()
        model.get()
        print(loss.get())



tensor(0.3034, requires_grad=True)
tensor(0.5315, requires_grad=True)
tensor(0.2957, requires_grad=True)
tensor(0.5188, requires_grad=True)
tensor(0.2883, requires_grad=True)
tensor(0.5066, requires_grad=True)
tensor(0.2811, requires_grad=True)
tensor(0.4946, requires_grad=True)
tensor(0.2741, requires_grad=True)
tensor(0.4831, requires_grad=True)
tensor(0.2674, requires_grad=True)
tensor(0.4718, requires_grad=True)
tensor(0.2610, requires_grad=True)
tensor(0.4609, requires_grad=True)
tensor(0.2547, requires_grad=True)
tensor(0.4503, requires_grad=True)
tensor(0.2487, requires_grad=True)
tensor(0.4400, requires_grad=True)
tensor(0.2429, requires_grad=True)
tensor(0.4300, requires_grad=True)


In [106]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [107]:
train()

tensor(2.8527, requires_grad=True)
tensor(0.1374, requires_grad=True)
tensor(0.4849, requires_grad=True)
tensor(0.1627, requires_grad=True)
tensor(0.5221, requires_grad=True)
tensor(0.1744, requires_grad=True)
tensor(0.5174, requires_grad=True)
tensor(0.1736, requires_grad=True)
tensor(0.5099, requires_grad=True)
tensor(0.1721, requires_grad=True)
tensor(0.5028, requires_grad=True)
tensor(0.1708, requires_grad=True)
tensor(0.4961, requires_grad=True)
tensor(0.1696, requires_grad=True)
tensor(0.4898, requires_grad=True)
tensor(0.1686, requires_grad=True)
tensor(0.4839, requires_grad=True)
tensor(0.1676, requires_grad=True)
tensor(0.4783, requires_grad=True)
tensor(0.1668, requires_grad=True)
tensor(0.4731, requires_grad=True)
tensor(0.1660, requires_grad=True)
tensor(0.4682, requires_grad=True)
tensor(0.1654, requires_grad=True)
tensor(0.4636, requires_grad=True)
tensor(0.1647, requires_grad=True)
tensor(0.4592, requires_grad=True)
tensor(0.1642, requires_grad=True)
tensor(0.4551, requi

AttributeError: 'Linear' object has no attribute '_object'